Iniciamos importando as principais bibliotecas que serão usadas durante a execução.

In [ ]:
# Para os CSVs
import pandas as pd

In [ ]:
# Para o modelo usaremos FASTAI
from fastai.vision.all import *
from fastai.callback.tracker import SaveModelCallback

In [ ]:
# Para acessar os arquivos
import os

In [ ]:
# Salvamos o caminho padrão em uma variável:
path = Path('../input/cassava-leaf-disease-classification')

In [ ]:
df_train = pd.read_csv(path/'train.csv')
df_train.head()

In [ ]:
# Carregar o DataSet e redimensionar as imagens 
dls = ImageDataLoaders.from_df(df_train, path=path, folder='train_images', item_tfms=Resize(224), valid_pct=0.1, seed=42, bs=64, batch_tfms=aug_transforms())
dls.show_batch()

Após alguns testes iniciais e comparação entre os modelos Alexnet, Resnet50 e Resnet18, constatou-se que o modelo que apresentou melhores resultados nas primeiras épocas foi o Resnet18. Portanto, procederemos o treinamento de mais épocas com este modelo.

In [ ]:
# cria modelo e uma callback para verificar e salvar o melhor modelo após cada época
learn = cnn_learner(dls, resnet18, metrics=[error_rate,accuracy], pretrained=False, path='../Learner')
save_callback = SaveModelCallback(fname='best_model', with_opt=True, monitor='valid_loss', reset_on_fit=False) 
# o parâmetro monitor informa como se deve se verificar se um modelo é melhor que outro
# valid_loss - para menor loss no cjto de validação
#train_loss - para menor loss no cjto de treino

In [ ]:
# caso for reiniciar uma execução
# carregando os melhores pesos
#learn = learn.load('best_model')
# carregando os últimos pesos
#learn = learn.load('last_model')

São executadas **25** épocas de treinamento. Após estes treinamentos, o modelo começa a convergir e não apresenta melhoras consideráveis.

In [ ]:
# treinamento, verificando se obteve um melhor modelo a cada época e salvando caso for um melhor modelo como best_model
nr_epochs = 25
learn.fit(nr_epochs, cbs=save_callback)

In [ ]:
# plota um gráfico de loss
learn.recorder.plot_loss()

In [ ]:
# salva os ultimos pesos (não necessariamente o melhor)
learn.save('last_model')

In [ ]:
learn.save('best_model')

In [ ]:
learn.export("./export.pth")

In [ ]:
submission_df = pd.read_csv(path/'sample_submission.csv')

test_data_path = submission_df['image_id'].apply(lambda x: path/'test_images'/x)
tst_dl = learn.dls.test_dl(test_data_path)
predictions = learn.tta(dl = tst_dl, n=10)

submission_df['label'] = np.argmax(predictions[0],axis=1)
submission_df

In [ ]:
submission_df.to_csv('submission.csv',index=False)